## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,broken clouds,76.96,89,66,5.01,EC,2022-08-07 20:28:44
1,1,Greenville,35.6127,-77.3663,clear sky,91.56,57,0,11.50,US,2022-08-07 20:34:19
2,2,Jamestown,42.0970,-79.2353,clear sky,90.01,49,0,10.36,US,2022-08-07 20:28:37
3,3,Hobart,-42.8794,147.3294,few clouds,35.06,95,20,1.99,AU,2022-08-07 20:28:51
4,4,Upernavik,72.7868,-56.1549,broken clouds,42.37,83,78,8.23,GL,2022-08-07 20:30:02


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,broken clouds,76.96,89,66,5.01,EC,2022-08-07 20:28:44
8,8,Avarua,-21.2078,-159.7750,scattered clouds,73.45,64,40,17.27,CK,2022-08-07 20:30:18
10,10,Cabo San Lucas,22.8909,-109.9124,scattered clouds,89.19,71,40,12.66,MX,2022-08-07 20:30:39
12,12,Rikitea,-23.1203,-134.9692,overcast clouds,72.52,65,100,31.74,PF,2022-08-07 20:28:41
20,20,Aripuana,-9.1667,-60.6333,moderate rain,89.38,45,77,2.37,BR,2022-08-07 20:35:41
28,28,Lompoc,34.6391,-120.4579,clear sky,88.02,64,0,18.41,US,2022-08-07 20:30:00
29,29,Santa Marta,11.2408,-74.1990,scattered clouds,89.65,62,40,9.22,CO,2022-08-07 20:35:44
37,37,Vaini,-21.2000,-175.2000,few clouds,77.16,88,20,6.91,TO,2022-08-07 20:28:57
39,39,Fortuna,40.5982,-124.1573,overcast clouds,71.92,76,100,8.05,US,2022-08-07 20:35:48
40,40,Emba,48.8264,58.1442,overcast clouds,77.40,37,98,15.41,KZ,2022-08-07 20:35:48


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                293
City                   293
Lat                    293
Lng                    293
Current Description    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Country                293
Date                   293
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
8,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,
10,Cabo San Lucas,MX,89.19,scattered clouds,22.8909,-109.9124,
12,Rikitea,PF,72.52,overcast clouds,-23.1203,-134.9692,
20,Aripuana,BR,89.38,moderate rain,-9.1667,-60.6333,
28,Lompoc,US,88.02,clear sky,34.6391,-120.4579,
29,Santa Marta,CO,89.65,scattered clouds,11.2408,-74.1990,
37,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
39,Fortuna,US,71.92,overcast clouds,40.5982,-124.1573,
40,Emba,KZ,77.40,overcast clouds,48.8264,58.1442,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Cabo San Lucas,MX,89.19,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
12,Rikitea,PF,72.52,overcast clouds,-23.1203,-134.9692,People ThankYou
20,Aripuana,BR,89.38,moderate rain,-9.1667,-60.6333,


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.count()

City                   293
Country                293
Max Temp               293
Current Description    293
Lat                    293
Lng                    293
Hotel Name             293
dtype: int64

In [13]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
10,Cabo San Lucas,MX,89.19,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
12,Rikitea,PF,72.52,overcast clouds,-23.1203,-134.9692,People ThankYou
20,Aripuana,BR,89.38,moderate rain,-9.1667,-60.6333,


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))